In [54]:
# Install Necessary Packages
!pip install openai pydantic --quiet

In [55]:
#Import necessary libraries
import os
import json
from pydantic import BaseModel, ValidationError
from typing import List
from openai import AzureOpenAI

In [56]:

# Set your Azure OpenAI credentials
AZURE_OPENAI_API_KEY="04f9a983b5d747baac8c74a75c0d525a"
AZURE_OPENAI_ENDPOINT="https://swedencentral.api.cognitive.microsoft.com/"
OPENAI_API_VERSION="2024-02-15-preview"

In [57]:
# Create AzureOpenAI client
client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version="2024-12-01-preview",
    azure_endpoint=AZURE_OPENAI_ENDPOINT
    )

In [58]:
#  Define schema using Pydantic
class MeetingNotes(BaseModel):
    summary: str
    action_items: List[str]

In [59]:
# Prompt template
PROMPT_TEMPLATE = """You are a meeting assistant.
1. Summarize the meeting transcript below in exactly two sentences.
2. Then list all action items mentioned, each as a separate bullet beginning with a dash.
Return the result strictly as JSON with keys "summary" and "action_items".

Transcript:
{transcript}
"""


In [60]:
# Main notes extraction function
def extract_meeting_notes(transcript: str) -> dict:
    prompt = PROMPT_TEMPLATE.format(transcript=transcript)

    messages = [{"role": "user", "content": prompt}]

    # First attempt
    response = client.chat.completions.create(
        model="telcogpt",
        messages=messages,
        temperature=0
    )

    result_text = response.choices[0].message.content.strip()

    try:
        return MeetingNotes.model_validate_json(result_text).model_dump()
    except (ValidationError, json.JSONDecodeError):
        # Retry with stricter system prompt
        retry_messages = [
            {"role": "system", "content": "Please output valid JSON only."},
            {"role": "user", "content": prompt}
        ]
        retry_response = client.chat.completions.create(
            model="telcogpt",
            messages=retry_messages,
            temperature=0
        )
        retry_text = retry_response.choices[0].message.content.strip()
        return MeetingNotes.model_validate_json(retry_text).model_dump()



In [61]:
# Sample input
sample_transcript = """Alice: Let's go over the Q2 results. We did well on the new feature rollout but need to address customer support backlog.
Bob: Yes, we should hire a new support engineer. Carol, can you take the lead on drafting the job description?
Carol: Absolutely, I'll have it ready by next Wednesday.
David: Also, marketing wants to update the feature page based on user feedback.
Alice: Great. So action items: Carol drafts JD, David coordinates with marketing, we reconvene next week.

"""


In [62]:
# Run the extraction
result = extract_meeting_notes(sample_transcript)
print(json.dumps(result, indent=2))

{
  "summary": "The team reviewed the Q2 results, noting success in the new feature rollout but highlighting the need to address the customer support backlog. Action items were assigned to Carol for drafting a job description for a new support engineer and to David for coordinating with marketing on updating the feature page.",
  "action_items": [
    "- Carol drafts the job description for a new support engineer.",
    "- David coordinates with marketing to update the feature page."
  ]
}
